# Uma Introdução ao Algoritmo Apriori

Este material utiliza partes do pos de Bernardo Costa no site Medium [Link]("https://medium.com/@bernardo.costa/uma-introdu%C3%A7%C3%A3o-ao-algoritmo-apriori-60b11293aa5a")  
Também da documentação da biblioteca [mlxtend]('http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/#frequent-itemsets-via-apriori-algorithm')

## 1. Introdução
Entre quase todas as listas dos **"Algoritmos que todo Data Scientist deve conhecer"**, o Apriori é um dos algoritmos que menos ouço falar, tanto em cursos quanto em canais de propagação de Data Science, motivo deste ser o primeiro algoritmo que apresento aqui no Medium.  
Antes de começar, preciso contar um pouco mais sobre a abordagem de apresentação que farei aqui nesse texto. 
Primeiro eu vou apresentar a visão mais intuitiva do algoritmo, ou seja, qual o tipo de problema ele se dispõe a resolver e a lógica usada para tal.  
Depois apresentarei uma visão mais matemática, mostrando as "contas" por trás da coisa, e por fim, mostrarei um um exemplo aplicado em um dataset de amostra, utilizando Python.

## 2. Visão Intuitiva do Apriori
## 2.1 Intuição de Regras de Associação

![image](https://miro.medium.com/max/1400/1*Mfmf9aDE_SvWBvrTRhQdrQ.jpeg)

Para entender melhor o que o Apriori tenta resolver. Imagine que você está trabalhando como vendedor de um supermercado e deseja descobrir algum método para aumentar suas vendas, aumentando assim os lucros. Para isso você resolve tentar "induzir" seu cliente a comprar produtos que ele vá consumir ao mesmo tempo, colocando-os próximos nos mercados. 

Com essa ideia em mente, você decide testar sua hipótese com os produtos do dia-a-dia dos brasileiros, o pão e a manteiga. Para fazer isso você move as prateleiras onde os pães ficam para próximo de onde a manteiga fica. Passado um tempo, você percebe que os clientes estão comprando mais pão e manteiga juntos.  

Com o sucesso das vendas do pão e da manteiga, você decide explorar mais essa ideia de produtos que podem ser comprados juntos. No meio das suas pesquisas, descobre que [fraldas e cervejas]("https://www.forbes.com/forbes/1998/0406/6107128a.html#24b642016260) são produtos que saem bastante juntos. Sem entender muito bem o motivo disso, você rapidamente coloca a prateleira das fraldas perto da prateleira d cerveja, fazendo a ilação de que pais estressados com seus bebês tendem a comprar cerveja para relaxar, o que faz suas vendas dispararem.  

Dizem que essa história aconteceu no Walmart e faz parte das lendas de Data Mining. Lenda ou não, ela é uma boa história pra contar para os parentes no final do ano e entender melhor o conceito de [Regras de Associação - Association Rules](https://en.wikipedia.org/wiki/Association_rule_learning).

## 2.2 Intuição do Algoritmo Apriori  
Para dar uma visão intuitiva do que o Apriori faz, iremos trabalhar com a amostra de um conjunto de dados referente a transações em uma loja de conveniência fictícia. Cada linha da amostra contém uma transação identificada por um ID, onde se a coluna referente ao produto (Cerveja, Fralda, Chiclete, Refrigerante e Salgadinho) estiver marcada com 1 significa que esse produto foi comprado nessa transação, 0 caso o contrário:
![Amostra do Conjunto de Dados](https://miro.medium.com/max/638/1*CMbFdBkQPbkaF_D_w1Y1rw.png)

## Regras de Associação
Com isso entendido, podemos definir então o que seria uma Regra de Associação, que nada mais é que um método de explorar relações entre Itens em conjuntos de dados. Vamos definir os seguintes termos para Regra de Associação:
- 1. I (Itens): conjunto dos seus n atributos {i_1, i_2, …, i_n};
- 2. D (Database): conjunto das m transações {t_1, t_2, …, t_m};
- 3. Toda transação t_i, é única em D e consiste em um subconjunto dos Itens I;
- 4. Vamos Definir uma Regra de Associação como a relação (X => Y) , onde X e Y são subconjuntos de I. Eles não podem ter nenhum elemento em comum.
- 5. X é chamado de antecedente e Y da consequência da Regra.  

Com isso definido, considere a transação de ID igual a 2, podemos ter a seguinte Regra de Associação: {Beer} => {Diaper}. Essa é a Regra de Associação que define que nessa transação, quem comprou Cerveja também comprou Fralda.  
Nesse ponto, se você começar a montar as Regras de Associação na sua cabeça, perceberá que mesmo para um conjunto de dados pequeno, existem muitas Regras.  
O desafio agora é encontrar um modo de selecionar as regras relevantes. 
O modo utilizado para encontrar tais regras envolve o cálculo de medidas como **Support, Confidence, Lift e Conviction**.
Embora o Apriori use apenas o Support, falarei um pouco dessas quatro para maior entendimento do problema que estamos tentando resolver.

### Support (Suporte)
A medida que indica a proporção de X em D.
$$
supp(X) = (qtd.X em D)/(qtd. D)
$$
no exemplo $supp(BEER) = 4/9$

### Confidence (Confiança)
A medida de Confidence é calculada em cima de uma Regra (X => Y). Ela expressa a proporção de "Se X for comprado, qual a chance de Y ser comprado?"  
$$Conf(X=>Y)=Supp(X U Y)/supp(X)$$
Ou seja, $Conf({Beer}=>{Gum}) = (2/9)/(4/9) = 1/2$.

### Lift (Levantamento, Alavancagem , ou algo assim)
A medida de Lift indica qual a chance de Y ser comprado, se X for comprado, e considerando toda a popularidade de Y. Em outras palavras, veja qual informação nos trás mais conhecimento sobre a possibilidade de Itens serem comprados juntos: 1) Conf(X) é muito alta se ele aparece em todas as transações ou 2) Conf(X) é alta e ele aparece em poucas transações? Essa diferenciação é calculada pelo Lift.
$$Lift(X=>Y)=Supp(X U Y)/(supp(X)*supp(X))$$  
No caso da Regra $({Beer}=>{Gum}), Lift({Beer}=>{Gum}) = (2/9)/((4/9)*(2/9)) = 2/3$.

Com base nesse valor conferirmos:  
Se $Lift(X=>Y) > 1$, então o conjunto Y é provável de ser comprado quando X for comprado.  
Se $Lift(X=>Y) ≤1$, então NÃO é provável que Y seja comprado, caso X seja comprado.

### Conviction (Convicção)
A medida de Conviction está interessada em calcular a frequência que X ocorre e Y não ocorre, ou seja, ela está interessada em quando a Regra falha.
$$
Conv(X=>Y) = (1 — Supp(Y)) / (1 — Conf(X=>Y))
$$

Essa medida varia entre [0, inf]. Se Conf(X=>Y) for igual a 1, então o denominador da fórmula é zerado e o resultado do Convinction é definido como infinito. Já se o Supp(Y) for igual a 1, ou seja, Y é presente em todas as transações, então Conv é igual a 0 — Você não erra nunca.  

Vamos para o Exemplo:  
Para facilitar a escrita aqui no texto, considere $X = {Soda, Snack} e Y = {Beer}$.
Precisamos primeiro calcular $Supp(Y) = 4/9$.  
Depois calculamos a $Conf(X=>Y) = 3/9$.  
Repare que transformei 1 em 9/9 para facilitar as contas.  
$Conv(X=>Y) = ((9/9) — (4/9)) / ((9/9) — (3/9))$  
$Conv(X=>Y) = (5/9) / (6/9)$  
$Conv(X=>Y) = (5/9) / (6/9)$  
$Conv(X=>Y) = 5/6$

### Apriori
Com entendimento das Regras de Associação, e de como interpretá-las, chegou a hora de entender o Apriori e como ele nos ajuda nesse processo de Data Mining.  
O Apriori trabalha com o conceito de Itens frequentes, que são os Itens do seu conjunto I que têm a pontuação do Support mais que um threshold (hiper-parâmetro). 
Ou Seja, precisamos calcular o Support de todas as combinações de Itens e extrair um subconjunto de Itens frequentes. Sabendo disso, os passos do Apriori são:  
- 1.Dentro dos Itens I, extraia um subconjunto (I_freq) dos Itens que tem o seu Support maior que o threshold.
- 2.Dentro de I_freq, itere para formar as combinações dos I_freq com I, aplique o threshold e acumule em I_freq.
- 3.Pare quando ao aplicar o threshold nenhum item sobrar.  
Por exemplo, considere o nosso conjunto de dados de amostra e threshold=0.4:  
#### 1. Vamos calcular o Support para grupos de 1 Item:
![Amostra do Conjunto de Dados](https://miro.medium.com/max/311/1*nnZoQ_waUZMXAoPD9K9INQ.png "Cálculo do support para 1 Item")  
#### 2. Agora removemos os Items com Support menor que o threshold, ou seja, sai o Chiclete (note que esse é o primeiro I_freq)  
![Amostra do Conjunto de Dados](https://miro.medium.com/max/315/1*Ja3Dp3Uz8ytO1cGXvoXsqQ.png "Cálculo do support para 1 Item") 
#### 3. Depois disso, realizamos o mesmo cálculo pra os pares de Itens, como por exemplo (Diaper, Beer)
![Amostra do Conjunto de Dados](https://miro.medium.com/max/345/1*Bzt1gNhIKGYCKqBgoFVRyg.png "Cálculo do support para 1 Item") 
#### 4. Aplicando o threshold, sobra apenas (Diaper, Beer), que devemos acumular em I_freq:
![Amostra do Conjunto de Dados](https://miro.medium.com/max/335/1*Hx2OUR7eUzOjNxqinkqJcA.png)  
Veja o I_freq do momento:
![Amostra do Conjunto de Dados](https://miro.medium.com/max/341/1*-hpTttkLNqO7UFQMWF_nXw.png)  
#### 5. Calculamos agora para 3 items:
![Amostra do Conjunto de Dados](https://miro.medium.com/max/632/1*gTRecfAqqRuRtoAk7R14nQ.png)
#### 6. Após aplicar o threshold, percebemos que nenhum Item sobrou, logo o algoritmo para, visto que esse é o critério de parada.  
![Amostra do Conjunto de Dados](https://miro.medium.com/max/341/1*-hpTttkLNqO7UFQMWF_nXw.png)

Pronto, encontramos as Regras de Associação mais frequentes. Olhando o resultado acima, você colocaria as Fraldas perto das Cerveja?

# A biblioteca mlextend

### Conjuntos de itens frequentes por meio do algoritmo Apriori
#### Função Apriori para extrair conjuntos de itens frequentes para mineração de regras de associação

from mlxtend.frequent_patterns import apriori

#### Visão geral
Apriori é um algoritmo popular [Ref 1 Neste Link]("https://www.it.uu.se/edu/course/homepage/infoutv/ht08/vldb94_rj.pdf") para extrair conjuntos de itens frequentes com aplicativos no aprendizado de regras de associação. O algoritmo apriori foi projetado para operar em bancos de dados contendo transações, como compras de clientes de uma loja. Um conjunto de itens é considerado "frequente" se atingir um limite de suporte especificado pelo usuário. Por exemplo, se o limite de suporte for definido como 0,5 (50%), um conjunto de itens frequentes será definido como um conjunto de itens que ocorrem juntos em pelo menos 50% de todas as transações no banco de dados.

#### Referências
[Ref 1 Neste Link]("https://www.it.uu.se/edu/course/homepage/infoutv/ht08/vldb94_rj.pdf") Agrawal, Rakesh, and Ramakrishnan Srikant. "Fast algorithms for mining association rules." Proc. 20th int. conf. very large data bases, VLDB. Vol. 1215. 1994.

### Instalando o mlxtend

In [1]:
import sys
!{sys.executable} -m pip install mlxtend

## Exemplo 1 - Gerando conjuntos de itens frequentes
A função apriori espera dados em um DataFrame de pandas codificado com um hot hot. Suponha que tenhamos os seguintes dados de transação:

In [2]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

Podemos transformá-lo no formato correto através do `TransactionEncoder` da seguinte maneira:

In [4]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


Agora, vamos devolver os itens e conjuntos de itens com pelo menos 60% de suporte:

In [5]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


Por padrão, `apriori` retorna os índices da coluna dos itens, que podem ser úteis em operações posteriores, como a mineração de regras de associação. Para melhor legibilidade, podemos definir use_colnames = True para converter esses valores inteiros nos respectivos nomes de itens:

In [6]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Yogurt, Kidney Beans)"


## Exemplo 2 - Selecionando e filtrando resultados  
A vantagem de trabalhar com os DataFrames do pandas é que podemos usar seus recursos convenientes para filtrar os resultados. Por exemplo, suponha que estamos interessados apenas em conjuntos de itens de comprimento 2 que tenham suporte de pelo menos 80%. Primeiro, criamos os conjuntos de itens frequentes via apriori e adicionamos uma nova coluna que armazena o comprimento de cada conjunto de itens:

In [7]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.8,(Eggs),1
1,1.0,(Kidney Beans),1
2,0.6,(Milk),1
3,0.6,(Onion),1
4,0.6,(Yogurt),1
5,0.8,"(Eggs, Kidney Beans)",2
6,0.6,"(Eggs, Onion)",2
7,0.6,"(Milk, Kidney Beans)",2
8,0.6,"(Onion, Kidney Beans)",2
9,0.6,"(Yogurt, Kidney Beans)",2


Em seguida, podemos selecionar os resultados que atendem aos critérios desejados da seguinte maneira:

In [8]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length
5,0.8,"(Eggs, Kidney Beans)",2


: Da mesma forma, usando a API do Pandas, podemos selecionar entradas com base na coluna "itemsets":

In [9]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

,support,itemsets,length
6,0.6,"(Eggs, Onion)",2


## Frozensets

Observe que as entradas na coluna "itemsets" são do tipo frozenset, que é do tipo Python interno semelhante a um conjunto de Python, mas imutável, o que o torna mais eficiente para determinadas operações de consulta ou [comparação](https://docs.python.org/3.6/library/stdtypes.html#frozenset). Como frozensets são conjuntos, a ordem dos itens não importa. Ou seja, a consulta

frequent_itemsets [frequent_itemsets ['itemsets'] == {'Cebola', 'Ovos'}]

é equivalente a qualquer um dos três seguintes

frequent_itemsets [frequent_itemsets ['itemsets'] == {'Ovos', 'Cebola'}]
frequent_itemsets [frequent_itemsets ['itemsets'] == frozenset (('Ovos', 'Cebola'))]
frequent_itemsets [frequent_itemsets ['itemsets'] == frozenset (('Cebola', 'Ovos'))]